<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="./images/btp-banner.gif" alt="SAP HANA Cloud">
</div>

## Retrieval-Augmented Generation with SAP HANA Cloud Vector Engine

In this demo, we will explore how to enhance the capabilities of Large Language Models (LLMs) with **SAP HANA Cloud vector engine**. You will learn how to embed unstructured and semi-structured data data using LLMs from **SAP Generative AI Hub**, store the vecotr embeddings in **SAP HANA Cloud**. Additionally, you will query vector embeddings, and forward the relevant results to a LLM for a augmented answer. 

## 🎯Learning Objectives
By the end of this demo, you will be able to:
- Implement a full RAG pipeline using Python, LangChain, and Generative AI Hub SDK.
- Generate embeddings for document chunks using Generative AI Hub SDK.
- Retrieve the most relevant content based on semantic similarity using SAP HANA Cloud similarity search.
- Augment user prompts with retrieved context and invoke LLMs (e.g., GPT-4o) to generate more accurate, grounded answers.
- Design and use prompt templates to enhance the quality of generated responses.

## 🚨Requirements

Please review the following requirements before starting the demo: 
- Enable the additional feature **Natrual Language Processing (NPL)** in your SAP HANA Cloud database 
- Deploy Large Language Models (LLMs): **text-embedding-ada-002** and **gpt-4o** in SAP AI Launchpad

## 📝About the Data

The data set is a product catalog of IT accessory products. Here are the main attributes and their descriptions based on the sample data:

|Field          |Description            |
----------------|-----------------------
|**PRODUCT_ID**| A unique identifier for each product.|
|**PRODUCT_NAME**| The name of the product, which typically includes the brand and the model.|
|**CATEGORY**| The general category of the product, which is "IT Accessories" for all entries sampled.|
|**DESCRIPTION**| A detailed description of the product, highlighting key features and specifications.|
|**UNIT_PRICE**| The price of the product in Euros.|
|**UNIT_MEASURE**| The unit of measure for the product, typically "Each" indicating pricing per item.|
|**SUPPLIER_ID**| A unique identifier for the supplier of the product.|
|**SUPPLIER_NAME**| The name of the supplier.|
|**LEAD_TIME_DAYS**| The number of days it takes from order to delivery.|
|**MIN_ORDER**| The minimum order quantity required.|
|**CURRENCY**| The currency of the transaction, which is "EURO" for all entries.|
|**SUPPLIER_COUNTRY**| The country where the supplier is located, which is "Germany" for all sampled entries.|
|**SUPPLIER_ADDRESS**| The physical address of the supplier.|
|**AVAILABILITY_DAYS**| The number of days the product is available for delivery.|
|**SUPPLIER_CITY**| The city where the supplier is located.|
|**STOCK_QUANTITY**| The quantity of the product currently in stock.|
|**MANUFACTURER**| The company that manufactured the product.|
|**CITY_LAT**| Geographical coordinates of the city (latitude)|
|**CITY_LONG**| Geographical coordinates of the city (longitude).|
|**RATING:**| A rating for the product, which are on a scale from 1 to 5.|

</br>

This dataset is structured to support various business operations such as inventory management, order processing, and logistics planning, providing a comprehensive view of product offerings, supplier details, and stock levels. Each entry is highly detailed, suggesting the dataset could be used for analytical purposes, such as optimizing supply chain operations or analyzing sales and marketing strategies.

</br>


### Step 1: Install Python packages

Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes.

**Note:** Jupyter Notebook kernel restart required after package installation.

In [ ]:
%pip install hdbcli --break-system-packages
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install langchain-hana --break-system-packages
%pip install folium --break-system-packages
%pip install ipywidgets --break-system-packages
%pip install python-dotenv --break-system-packages

# kernel restart required!!!

⚠️The Python kernel needs to be restarted before continuing. 

> ![](./images/config_001.png)

### Step 2: Configure AI Core Client
Execute the configuration module below to enable access to SAP Generative AI foundation models. This configuration is automatically done by running configuration module in the code block.

> You could also set up the same by running a terminal command:
>```bash
>aicore configure
>```

In [ ]:
# Generative AI Config
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

# Get the AI Core URL from environment variables
URL = os.getenv('AICORE_AUTH_URL')
# Get the AI Core client ID from environment variables
CLIENT_ID = os.getenv('AICORE_CLIENT_ID')
# Get the AI Core client secret from environment variables
CLIENT_SECRET = os.getenv('AICORE_CLIENT_SECRET')
# Get the AI Core client ID from environment variables
RESOURCE_GROUP = os.getenv('AICORE_RESOURCE_GROUP')
# Get the AI Core client secret from environment variables
API_URL = os.getenv('AICORE_BASE_URL')

# Set up the AICoreV2Client
ai_core_client = AICoreV2Client(base_url=API_URL,
                            auth_url=URL,
                            client_id=CLIENT_ID,
                            client_secret=CLIENT_SECRET,
                            resource_group=RESOURCE_GROUP)

# Initialize GenAIHub proxy client
proxy_client = GenAIHubProxyClient(ai_core_client=ai_core_client)
print("✅AI Core Client connection is established successfully!")

Run the test below to test SAP AI Core configuration by making a call to the **text-embedding-ada-002** model.  

In [ ]:
# Test embeddings

from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="SAP Generative AI Hub is awesome!",
    model_name="text-embedding-ada-002"
    
)
print(response.data)

Initialize the embeddings model **text-embedding-ada-002**. This model will be used to generate embeddings for both documents and the user's prompts.

In [ ]:
# Initialize embeddings

from gen_ai_hub.proxy.langchain import OpenAIEmbeddings
open_ai_embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002', chunk_size=100, max_retries=10)
print("✅Embedding model 'text-embedding-ada-002' is initialized successfully!")

Initialize the LLM model **gpt-4o**. This model is used for generating responses or interacting in a chat-like environment.

In [ ]:
# Initialize LLM

from gen_ai_hub.proxy.langchain import ChatOpenAI
llm = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)
print("✅LLM model 'gpt-4o' is initialized successfully!")

### Step 3: Ask LLM without business context

After completing the configuration we are ready to ask the first question directly to LLM (gpt-4o) without any business product context to find us products with a rating of 4 and more. The response is arbitrary and does not relate to our product data. 

>**Note**: If we directly pass the prompt to the model without RAG, this will be the output. Later, we can compare how the output produced by RAG is different from this output. 

In [ ]:
from IPython.display import Markdown

question = "Suggest a keyboard with a rating 4 or more"
llm.temperature=0.7
response = llm.invoke(question)
display(Markdown(response.content))

### Step 4: Prepare the documentation

This code snippet demonstrates how to load and process text data from a CSV file using the `CSVLoader` from the `langchain.document_loaders.csv_loader` module.

This process is useful for handling large text data, making it more manageable or suitable for further processing, analysis, or input into machine learning models, especially when dealing with limitations on input size.



In [ ]:
# Process CSV data file

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path="data/new_product.csv",
    encoding="utf-8",
    csv_args={
        "delimiter": ";",
        "quotechar": '"'
    },
)

# Process data

text_documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_chunks = text_splitter.split_documents(text_documents)
print(f"Number of document chunks: {len(text_chunks)}")
# print(text_chunks)

for chunks in text_chunks:
    print(chunks.metadata)
    print(chunks.page_content)

### Step 5: Connect to SAP HANA Cloud Database

The provided Python script imports database connection modules and initiates a connection to a SAP HANA Cloud instance using the `dbapi` module. The user is prompted to enter their username and password, which are then used to establish a secure connection to the SAP HANA Cloud database. 

In [ ]:
# HC Vector Engine
from hdbcli import dbapi

# Get the HANA Cloud username from environment variables
HANA_USER = os.getenv('HANA_VECTOR_USER')
# Get the HANA Cloud password from environment variables
HANA_PASS = os.getenv('HANA_VECTOR_PASS')
# Get the HANA Cloud host from environment variables
HANA_HOST = os.getenv('HANA_VECTOR_HOST')

# Establish a connection to the HANA Cloud database using HANA_ML package
connection = dbapi.connect(
    address=HANA_HOST,
    port=443,
    user=HANA_USER,
    password=HANA_PASS,
    autocommit=True,
    sslValidateCertificate=False,
)
print("✅HANA Cloud connection is established successfully!")

### Step 6: Create a LangChain VectorStore interface to store the embeddings

The `langchain_community.vectorstores.hanavector` library, specifically the `HanaDB` class, from the LangChain community, is designed to interact with vector data stored in SAP HANA Cloud database, and enables developers to utilize SAP HANA Cloud's advanced capabilities for managing and querying vector data, in the context of AI and machine learning applications.

This cell will create a LangChain VectorStore interface for the HANA database and specify the table (collection) to use for accessing the vector embeddings. Embeddings are vector representations of text data that incorporate the semantic meaning of the text.

In [ ]:
from langchain_hana import HanaDB

#Create a LangChain VectorStore interface for the HANA database and specify the table (collection) to use for accessing the vector embeddings
vector_table = HanaDB(
    embedding=open_ai_embedding_model, 
    connection=connection, 
    table_name="PRODUCTS_IT_ACCESSORY_ADA_"+ HANA_USER,
    content_column="VEC_TEXT", # the original text description of the product details
    metadata_column="VEC_META", # metadata associated with the product details
    vector_column="VEC_VECTOR" # the vector representation of each product 
)

print("✅Vector Database connection is established successfully!")

In [ ]:
# Delete already existing documents from the table
vector_table.delete(filter={})

# add the loaded document chunks
vector_table.add_documents(text_chunks)
print("✅The vector table is created successfully!")

In [ ]:
# Query the table to verify embeddings
cursor = connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{vector_table.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

for vector in vectors:
    print(vector)

### Step 7: Define a prompt template to provide business context

Define a prompt template to provide context to our prompts. Thus, when passed to the model, the template will add the necessary context to the prompt so that more accurate results are generated.

The answer should contain the requested information about products and their descriptions, formatted according to the specified JSON structure for further use in the SAP HANA JSON Document store.

> **Note**: The created template for the prompt contains two variables - **context** and **question**. These variables will be replaced with the context and question in the upcoming steps.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """use the following pieces of context to answer the question at the end. If you don't know the answer,
    just say you don't know, don't try to make up an answer. Format the results in a list of JSON items with the following keys:

        "PRODUCT_ID", 
        "PRODUCT_NAME",
        "CATEGORY",
        "DESCRIPTION",
        "UNIT_PRICE",
        "UNIT_MEASURE",
        "SUPPLIER_ID",
        "SUPPLIER_NAME",
        "LEAD_TIME_DAYS",
        "MIN_ORDER",
        "CURRENCY",
        "SUPPLIER_COUNTRY",
        "SUPPLIER_ADDRESS",
        "SUPPLIER_CITY",
        "CITY_LAT",
        "CITY_LONG",
        "RATING"
      
    
    The 'RATING' key value is an integer datatype ranging from 0 stars to 5 stars. Where 0 stars is 'bad' and 5 stars is 'excellent'. Do not include json markdown codeblock syntax in the results for example: ```json ```

    {context}

    question: {question}

    """


PROMPT = PromptTemplate(template = prompt_template, 
                        input_variables=["context", "question"]
                       )
    
chain_type_kwargs = {"prompt": PROMPT}

### Step 8: Generate more accurate, grounded answers with retrieved context 
This code snippet integrates various components from the `langchain` library to create a retrieval-based question-answering (QA) system. Here's a breakdown of the key parts and their functionality:

- **Retriever Initialization:** The `db.as_retriever` function is used to initialize a retriever object with specific search arguments (`'k':20`), which likely defines the number of search results to consider.

- **Prompt Template :** The `PromptTemplate` was defined in the previous step that instructs how to use the context to answer a question. It emphasizes not to fabricate answers if the information is unavailable. The template also outlines the structure for the expected JSON output with various product and supplier details.

In [ ]:
question = "Find products with a rating of 4 and more."
retriever = vector_table.as_retriever(search_kwargs={'k':20})

qa = RetrievalQA.from_chain_type(llm=llm,
                 retriever=retriever, 
                 chain_type="stuff",
                 chain_type_kwargs= chain_type_kwargs)

answer = qa.run(question)
print(answer)
print("💯This is the end of the demo. Thank you for your attention!")

## 🚧Optional Exercise: SAP HANA Cloud multi-modeling with Spatial and Vector engines
In this demo, we will explore how to combine the vector embeddings with spatial data within the same SQL environment, which is powerful especially for building intelligent applications. You will learn how to define a table including both vector embeddings and spatial data in **SAP HANA Cloud**. At the end, you will visualize the warehouse locations on a map.  

## 🎯Learning Objectives
By the end of this demo, you will be able to:
- Understand the data types: vector embeddings and spatial data. 
- Define a table for both vector embeddings and spatial data.
- Visualize the results on a map.

## 🚨Requirements

Please review the following requirements before starting the demo: 
- Enable the additional feature **Document Store** in your SAP HANA Cloud database
- Complete the previous demo

### Step 1: Save RAG results to SAP HANA Cloud Document Store (JSON)

Create a document collection **GX_RAG_PRODUCTS_DEV** to store the RAG results from the previous step. This script iterates over the list of product dictionaries, converting each one into a JSON string before inserting it into the collection. It then queries the collection to fetch and print all the inserted documents, allowing you to verify the insertion.

In [ ]:
# Sample dataset of product details
import json

jdata = json.loads(answer)
products = jdata
collection_name = "GX_RAG_PRODUCTS_DEV"

# Create a cursor
cursor = connection.cursor()

# Create a collection (document store)
try:
    cursor.execute(f"CREATE COLLECTION {collection_name}")
    print(f"{collection_name} Created!")
    
    # Insert JSON data into the collection
    for product in products:
        json_str = json.dumps(product)
        cursor.execute(f"INSERT INTO {collection_name} VALUES ('{json_str}')")

except :
    print(f"{collection_name} Recreated!")
    cursor.execute(f"DROP COLLECTION {collection_name}")
    cursor.execute(f"CREATE COLLECTION {collection_name}")

    # Insert JSON data into the collection
    for product in products:
        json_str = json.dumps(product)
        cursor.execute(f"INSERT INTO {collection_name} VALUES ('{json_str}')")




This code is designed to validate the created collection **GX_RAG_PRODUCTS_DEV** by retrieving supplier information, including their ID, city, and location represented as a geospatial point.

In [ ]:
collection_name = "GX_RAG_PRODUCTS_DEV"
sql = f"SELECT SUPPLIER_ID, SUPPLIER_CITY,SUPPLIER_ADDRESS,NEW ST_POINT(TO_DOUBLE(CITY_LONG),TO_DOUBLE(CITY_LAT)) AS SUPPLIER_LOCATION FROM {collection_name}"

cursor.execute(sql)
suppliers = cursor.fetchall()

for supplier in suppliers:
    print(supplier)

### Step 2: Create a new table including both vector embeddings and spatial data
This code snippet is designed to create a new table called **GX_SUPPLIER_LOCATIONS**, specifically for storing supplier location information. Note that **SUPPLIER_LOCATION** is a geospatial point representing the supplier's location, created using the `ST_POINT()` function. 

This point is constructed from two columns - **CITY_LONG** and **CITY_LAT** - which are converted to double precision numbers (representing longitude and latitude, respectively).



In [ ]:
table_name = "GX_SUPPLIER_LOCATIONS_DEV"
sql = f"""CREATE TABLE {table_name} AS (
               SELECT SUPPLIER_ID,
               SUPPLIER_CITY,
               NEW ST_POINT(
               TO_DOUBLE(CITY_LONG),
               TO_DOUBLE(CITY_LAT)) AS SUPPLIER_LOCATION,
               SUPPLIER_ADDRESS,
               PRODUCT_NAME,
               RATING

               FROM {collection_name})"""

try:
    cursor.execute(sql)
    print(f"{table_name} Created!")

except :
    print(f"{table_name} Recreated!")
    cursor.execute(f"DROP TABLE {table_name}")
    cursor.execute(sql)



### Step 3: Query new table to retrieve the geospatial data. 

This code snippet is designed to query SAP HANA Cloud using SQL to retrieve information about suppliers, including their IDs, city names, and geographic locations (latitude and longitude). Here's a breakdown of the code's functionality:

The latitude and longitude are extracted using the `ST_Y()` and `ST_X()` functions on the **SUPPLIER_LOCATION** column, which is presumably stored as a geospatial data type in the database.




In [ ]:
# Query to fetch the geospatial data
table_name = "GX_SUPPLIER_LOCATIONS_DEV"
sql = f"""SELECT SUPPLIER_ID,
               SUPPLIER_CITY,
               SUPPLIER_LOCATION.ST_Y() as latitudes, 
               SUPPLIER_LOCATION.ST_X() as longitudes,
               SUPPLIER_ADDRESS,
               PRODUCT_NAME,
               RATING 
            FROM {table_name}"""

cursor.execute(sql)

# Fetch all the results
points = cursor.fetchall()


for point in points:
    print(point)

# Close the cursor and connection
cursor.close()

### Step 4: Visualize supplier warehouse locations on a map 

The provided code snippet uses the `folium` library in Python to create an interactive map centered around Germany. It initializes a map object centered at the coordinates for Germany (latitude 51.1657, longitude 10.4515) with a zoom level of 6. Then, it iterates over a dataset named `data`, which is expected to contain points of interest. Each point in the dataset includes a supplier ID, city name, latitude, and longitude.

For each point, the code creates a popup text that includes the city name and supplier ID. It then creates a marker at the point's latitude and longitude, attaches the popup text to this marker, and adds the marker to the map. The result is an interactive map where each marker represents a point of interest, and clicking on a marker displays a popup with additional information about that point.

In [ ]:
import folium

data = points

# Create a map centered around the average location
average_lat = sum([item[2] for item in data]) / len(data)
average_lon = sum([item[3] for item in data]) / len(data)
map = folium.Map(location=[average_lat, average_lon], zoom_start=7)

# Loop through each item in data to create markers
for supplier_id, city, lat, lon, address, product, rating in data:
    popup_text = f"Supplier ID: {supplier_id}<br>"
    popup_text += f"City: {city}<br>"
    popup_text += f"Address: {address}<br>"
    popup_text += f"Product: {product}<br>"
    popup_text += f"Rating: {rating}"
    
    # Add a marker to the map
    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_text, max_width=300,min_width=300)
    ).add_to(map)

# Display the map
map
